# Portfolio selection

In [2]:
#### Imports ####
#################

### basics
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import copy


### yahoo finance
import yfinance as yf

### pandas
import pandas as pd
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix


### data optimization library
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage,risk_matrix
from pypfopt import plotting

In [70]:
#### Functions ####
###################
def data_from_names(stocks_names):
    '''
    Descarga y devuelve el conjunto de stocks ingresado con un periodo diario de 1d de los últimos 10 años.

    Argumentos:
    stock_names: (list) Nombres de los stocks en formato simbolo cómo están identificados en yahoo finance.
    '''
    dataset = pd.DataFrame()
    for s in stocks_names:
        value = yf.Ticker(s).history(period = "10y").Close
        dat = pd.DataFrame({"{}".format(s): value})
        dataset = pd.concat([dataset,dat],axis = 1)
    return dataset


def split_train_test(dataset,first,end):
    '''
    Divide el dataset en train y test. El train está destinado para hacer el modelo de portfolios y el test para hacer el backtest 
    
    Argumentos:
    dataset: (Dataframe) index:(Timestamp), cols:(float) stock price values
    first: ((list) shape=(3)) [(int) year,(int) month,(int) day]
    end:   ((list) shape=(3)) [(int) year,(int) month,(int) day]
    '''

    first = [str(f) for f in first]
    end = [str(e) for e in end]

    y_1,m_1,d_1 = first
    y_2,m_2,d_2 = end
    print( y_1,m_1,d_1)

    if len(m_1) == 1:
        m_1 = "0"+m_1
    if len(m_2) == 1:
        m_2 = "0"+m_2
    if len(d_1) == 1:
        d_1 = "0"+d_1
    if len(d_2) == 1:
        d_2 = "0"+d_2


    dates_times = dataset.index
    dates = [fulldate.strftime('%Y-%m-%d') for fulldate in dates_times]

    
    try:
        idx_1 =  dates.index(f"{y_1}-{m_1}-{d_1}")
    except:
        print("No existe esa fecha para el BEGIN, probar otra")

    try:
        idx_2 =  dates.index(f"{y_2}-{m_2}-{d_2}")
    except:
        print("No existe esa fecha para el END, probar otra")
    
    return dataset[idx_1:idx_2],dataset[idx_2:]


def data_cleaner(data):
    '''
    Divide el dataset en train y test. El train está destinado para hacer el modelo de portfolios y el test para hacer el backtest 
    
    Argumentos:
    dataset: (Dataframe) index:(Timestamp), cols:(float) stock price values
    first: ((list) shape=(3)) [(int) year,(int) month,(int) day]
    end:   ((list) shape=(3)) [(int) year,(int) month,(int) day]
    '''
    #Checking for any null values and removing the null values'''
    if data.isnull().values.any():
        print("Hay NULLS en el Dataframe")
        return

    else:
        print("Descarto las columnas que les faltan más del 30% de los datos")
        missing_fractions = data.isnull().mean().sort_values(ascending=False)
        missing_fractions.head(10)
        drop_list = sorted(list(missing_fractions[missing_fractions > 0.3].index))
        data.drop(labels=drop_list, axis=1, inplace=True)

        return





    

In [64]:
data = data_from_names(["ITOT","IVV","QQQ","VTI","IJR","VPL","VWO","SUSA","HYEM","MGV"])

In [65]:
train,test = split_train_test(data,[2014,10,3],[2014,10,13])

2014 10 3
2014 10 03


In [26]:
a = test.index

In [34]:
b = [fulldate.strftime('%Y-%m-%d') for fulldate in a]

In [41]:
b[np.where(b == '2013-03-11')[0][0]]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [48]:
np.where(b == '2013-03-11')

(array([], dtype=int64),)

In [52]:
b.index('2013-03-40')

ValueError: '2013-03-40' is not in list

In [71]:
data.shape
data_cleaner(data)
data.shape


Descarto las columnas que les faltan más del 30% de los datos


NameError: name 'dataset' is not defined

In [69]:
print('Null Values =',data.isnull().values.any())

Null Values = False
